In [6]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features
from src.visualization import visualize

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

DIAGS_RELEVANTES = [
    "C33X",
    "C340",
    "C341",
    "C342",
    "C343",
    "C381",
    "C384",
    "C450",
    "C780",
    "C782",
    "D143",
    "D381",
    "E848",
    "I080",
    "I081",
    "I340",
    "I350",
    "I351",
    "I352",
    "I420",
    "I456",
    "I472",
    "I495",
    "I710",
    "I712",
    "J398",
    "J47X",
    "J679",
    "J841",
    "J848",
    "J849",
    "J860",
    "J869",
    "J90X",
    "J91X",
    "J931",
    "J955",
    "J980",
    "M348",
    "Q211",
    "Q231",
    "Q676",
    "T820",
    "T821",
    "Z450",
]

DIAGS_FALTANTES_PULMON = ["C349", "C348"]

DIAGS_RELEVANTES += DIAGS_FALTANTES_PULMON

In [8]:
# Carga los egresos nacionales
egresos_nacionales = pd.read_csv(
    "../data/processed/ranking_nacional_egresos.csv", sep=";", encoding="latin-1"
)

In [9]:
# Filtra solamente los diagnosticos mas relevantes del Torax
egresos_mas_relevantes = (
    egresos_nacionales[egresos_nacionales["DIAG1"].isin(DIAGS_RELEVANTES)]
    .query("ANO_EGRESO >= 2017")
    .copy()
)

egresos_mas_relevantes["DIAG1"] = egresos_mas_relevantes["DIAG1"].replace(
    ["C340", "C341", "C342", "C343", "C348", "C349"], "C34"
)

In [10]:
# Obtiene cantidad de pacientes, egresos y dias de estada por diagnostico
resumen_egresos_mas_relevantes = pd.pivot_table(
    egresos_mas_relevantes,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_pacientes_distintos", "n_egresos", "dias_estada_totales"],
    aggfunc="sum",
    fill_value=0,
).sort_index()

In [11]:
# Importa los casos teoricos por cada problema de salud
casos_teoricos = pd.read_excel("../data/interim/casos_teoricos_diagnosticos.xlsx")
casos_teoricos["Diagnostico"] = casos_teoricos["Diagnostico"].str.split("-").str[0].str.strip()
casos_teoricos = casos_teoricos.set_index("Diagnostico")

# Obtiene los casos en el estrato nacional
casos_nacionales = casos_teoricos.query("Estrato == 'Pais' and Estadística == 'Incidencia'").copy()
casos_nacionales_del_2017_al_2020 = casos_nacionales[[f"Poblacion {i}" for i in range (2017, 2021)]]

In [12]:
# Une la cantidad de casos teoricos y reales
union_de_pacientes_teoricos_y_vistos = casos_nacionales_del_2017_al_2020.merge(
    resumen_egresos_mas_relevantes["n_pacientes_distintos"],
    how="left",
    left_index=True,
    right_index=True,
)

# Renombra las columnas
niveles_columnas = [
    ("poblacion_teorica", "2017"),
    ("poblacion_teorica", "2018"),
    ("poblacion_teorica", "2019"),
    ("poblacion_teorica", "2020"),
    ("poblacion_vista", "2017"),
    ("poblacion_vista", "2018"),
    ("poblacion_vista", "2019"),
    ("poblacion_vista", "2020"),
]

union_de_pacientes_teoricos_y_vistos.columns = pd.MultiIndex.from_tuples(niveles_columnas)

# Calcula la cantidad de cumplimiento de los casos teoricos
porcentaje_de_atencion_casos_teoricos = (
    union_de_pacientes_teoricos_y_vistos["poblacion_vista"].sum(axis=1)
    / union_de_pacientes_teoricos_y_vistos["poblacion_teorica"].sum(axis=1)
)

In [13]:
proyeccion_de_casos_a_atender = casos_nacionales[[f"Poblacion {i}" for i in range(2017, 2036)]].mul(
    porcentaje_de_atencion_casos_teoricos, axis=0
)

In [24]:
# Obtiene egresos por caso por diagnostico
egresos_por_persona_INT = resumen_egresos_mas_relevantes["n_egresos"].sum(
    axis=1
) / resumen_egresos_mas_relevantes["n_pacientes_distintos"].sum(axis=1)

# Obtiene dias de estada por egreso
dias_de_estada_por_egreso_INT = resumen_egresos_mas_relevantes["dias_estada_totales"].sum(
    axis=1
) / resumen_egresos_mas_relevantes["n_egresos"].sum(axis=1)

In [25]:
dias_de_estada_por_egreso_INT

DIAG1
C33X    12.297297
C34      9.342940
C381    10.062500
C384    10.573034
C450     7.659649
C780     9.110743
C782    10.046917
D143     5.785714
D381     7.998346
E848    14.449153
I080    15.369792
I081    18.123223
I340    14.297481
I350    11.722103
I351    12.418074
I352    13.955414
I420    11.225723
I456     2.851471
I472     7.919327
I495     5.703863
I710    12.450082
I712    11.577154
J398    13.526185
J47X    10.083333
J679    10.816901
J841     9.518958
J848     9.609091
J849     8.793794
J860    18.587121
J869    20.862876
J90X    11.140933
J91X     6.000000
J931     8.354894
J955    11.380655
J980     6.013587
M348    10.103896
Q211     4.930216
Q231     9.775000
Q676     4.951583
T820    13.197861
T821     5.481583
Z450     2.085503
dtype: float64